In [ ]:
%tensorflow_version 2.2
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import files
import time
from tensorflow.keras.utils import Sequence
import random
import math

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/PrivEdge/Dataset

/content/drive/MyDrive/PrivEdge/Dataset


In [ ]:
# Import libraries
import numpy as np 
import pandas as pd
import h5py

# Read the dataset for example h5 file
f = h5py.File('/content/drive/MyDrive/PrivEdge/Dataset/LetterColorImages_123.h5', 'r')
# List all groups
keys = list(f.keys())
print(keys)


# Create tensors (i.e. X) and targets (i.e. Y)
backgrounds = np.array(f[keys[0]])
tensors = np.array(f[keys[1]])
targets = np.array(f[keys[2]])
print ('Tensor shape:', tensors.shape)
print ('Target shape', targets.shape)
print ('Background shape:', backgrounds.shape)

# Save training and test tensors of each target separately for each user, (80,20) --> (Train, Test)
for i in set(targets):
    data=tensors[np.where(targets==i)]
    nP=len(data)
    idx_p = np.random.permutation(nP)
    training_idx_p, test_idx_p = idx_p[:int(nP*0.8)], idx_p[int(nP*0.2):]
    train = data[training_idx_p]
    test = data[test_idx_p]
    
    np.savez('user_%d'%(i), Xtr=train, Xte= test, label=np.repeat(i, len(tensors[np.where(targets==i)],)))

['backgrounds', 'images', 'labels']
Tensor shape: (14190, 32, 32, 3)
Target shape (14190,)
Background shape: (14190,)


In [ ]:
cd ..

/content/drive/My Drive/PrivEdge


In [ ]:
import scipy
from glob import glob
import numpy as np
import scipy.misc
class DataLoader():
    def __init__(self, img_res=(32, 32)):
        self.img_res = img_res
    
    def load_data(self,dataset_name,batch_size=128, is_testing=False):
        data_type = "Xtr" if not is_testing else "Xte" 
        path = '/content/drive/My Drive/PrivEdge/Dataset/%s.npz' %(dataset_name)
        data = np.load(path)
        traintest = data[data_type]
        if is_testing:
           batch_size=len(traintest)
        idx = np.random.choice((len(traintest)), size=batch_size)
        imgs= []
        for i in idx:
            img = traintest[i]
            imgs.append(img)

        imgs = np.array(imgs) / 127.5 - 1.

        return imgs

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [ ]:
from __future__ import print_function, division
import scipy
import errno
!pip install import_ipynb
import import_ipynb
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D,Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import sys
import numpy as np
import os


class RAN():
  def __init__(self,identity):
    self.identity=identity
    self.img_rows=32
    self.img_cols=32
    self.channels=3
    self.img_shape=(self.img_rows,self.img_cols,self.channels)
    self.dataset_name=self.identity
    self.data_loader=DataLoader(img_res=(self.img_rows,self.img_cols))
    patch=int(self.img_rows/2**4)
    self.disc_patch=(patch,patch,1)
    optimizer=Adam(0.0002,0.5)
    self.gf=32
    self.df=64
    self.discriminator=self.build_discriminator()
    self.discriminator.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
    self.reconstructor=self.build_reconstructor()
    print(self.reconstructor.summary())
    self.reconstructor.compile(loss='mse',optimizer=optimizer)
    img=Input(shape=self.img_shape)
    reconstr=self.reconstructor(img)
    self.discriminator.trainable=False
    valid=self.discriminator(reconstr)
    self.combined=Model(img,[reconstr,valid])
    self.combined.compile(loss=['mse','mse'],loss_weights=[0.999, 0.001],optimizer=optimizer)

  def build_reconstructor(self):
    
    def conv2d(layer_input,filters,f_size=4):
      
      d = Conv2D(filters,kernel_size=f_size,strides=2,padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
      d = InstanceNormalization()(d)
      return d

    def deconv2d(layer_input,filters,f_size=4,dropout_rate=0):
      
      u=UpSampling2D(size=2)(layer_input)
      u=Conv2D(filters,kernel_size=f_size,strides=1,padding='same',activation='relu')(u)
      if dropout_rate:
        u=Dropout(dropout_rate)(u)
      u=InstanceNormalization()(u)
      return u

    d0=Input(shape=self.img_shape)

    
    d1=conv2d(d0,self.gf)
    d2=conv2d(d1,self.gf*2)
    d3=conv2d(d2,self.gf*4)
    d4=conv2d(d3,self.gf*4)
    d5=conv2d(d4,self.gf*8)

    
    u1=deconv2d(d5,self.gf*8)
    u2=deconv2d(u1,self.gf*8)
    u3=deconv2d(u2,self.gf*8)
    u4=deconv2d(u3,self.gf*4)
    u5=deconv2d(u4,self.gf*2)

    output_img=Conv2D(self.channels,kernel_size=4,strides=1,padding='same',activation='tanh')(u5)
    return Model(d0,output_img)

  def build_discriminator(self):
    def d_layer(layer_input,filters,f_size=4,normalization=True):
      
      d=Conv2D(filters,kernel_size=f_size,strides=2,padding='same')(layer_input)
      d=LeakyReLU(alpha=0.2)(d)
      if normalization:
        d=InstanceNormalization()(d)
      return d
    
    img=Input(shape=self.img_shape)

    d1=d_layer(img,self.df,normalization=False)
    d2=d_layer(d1,self.df*2)
    d3=d_layer(d2,self.df*4)
    d4=d_layer(d3,self.df*8)

    validity=Conv2D(1,kernel_size=4,strides=1,padding='same')(d4)
    return Model(img,validity)

  def train(self,epochs,batch_size=128,save_interval=50):
    half_batch=int(batch_size/2)
    start_time=datetime.datetime.now()
    imgsVal=self.data_loader.load_data(self.identity,batch_size=half_batch,is_testing=True)
    #print(imgsVal.shape)
    TrainLoss=np.zeros(epochs)
    ValLoss=np.ones(epochs)
    for epoch in range(epochs):
      img=self.data_loader.load_data(self.identity,batch_size=half_batch)
      reconstr=self.reconstructor.predict(img)
      valid=np.ones((half_batch,)+self.disc_patch)
      fake=np.zeros((half_batch,)+self.disc_patch)

      d_loss_real=self.discriminator.train_on_batch(img,valid)
      d_loss_fake=self.discriminator.train_on_batch(reconstr,fake)
      d_loss=0.5*np.add(d_loss_real,d_loss_fake)

      img=self.data_loader.load_data(self.identity,batch_size=half_batch)
      #print(img.shape,imgsVal.shape,valid.shape)
      r_loss=self.combined.train_on_batch(img,[img,valid])
      r_loss_val=self.combined.test_on_batch(imgsVal[0:img.shape[0],:,:,:],[imgsVal[0:img.shape[0],:,:,:],valid])
      #r_loss=self.combined.train_on_batch(img,valid)
      #r_loss_val=self.combined.test_on_batch(imgsVal,valid)
      TrainLoss[epoch]=r_loss[0]
      print("imgValSize: {}\nvalid size: {}".format(imgsVal.shape, valid.shape))
      MinValLoss=ValLoss.min()
      ValLoss[epoch]=r_loss_val[0]
      print("%d [D loss: %f, acc.: %.2f%%] [R loss: %f] [R loss Val: %f] [Minimum: %f]" % (epoch,d_loss[0],100*d_loss[1],r_loss[0],r_loss_val[0],MinValLoss))

      if ValLoss[epoch]<MinValLoss and MinValLoss<0.04:
        self.save_imgs(epoch)
        self.reconstructor.save('SavedModel/%s/%s.h5'%(self.identity,self.identity))
      
      np.savez('loss/Loss_%s'%(self.dataset_name),TrLoss=TrainLoss,TeLoss=ValLoss) # create a empty folder 'loss' in 'PrivEdge'
  def save_imgs(self, epoch):
    r,c=2,2
    imgs=self.data_loader.load_data(self.identity,batch_size=1,is_testing=False)
    imgs_val=self.data_loader.load_data(self.identity,batch_size=1,is_testing=True)

    reconstr=self.reconstructor.predict(imgs)
    reconstr_val=self.reconstructor.predict(imgs_val)

    gen_imgs=np.concatenate([imgs, imgs_val, reconstr, reconstr_val])
    gen_imgs=0.5*gen_imgs+0.5

    titles=['Train','Val','Reconstructed']
    fig,axs=plt.subplots(r,c)
    cnt=0
    for i in range(r):
      for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt])
        axs[i, j].set_title(titles[j])
        axs[i,j].axis('off')
        cnt +=1  
      fig.savefig("ReconstructedImages/%s/TrainValSamples_E%d.png" %(self.dataset_name,epoch))
      plt.close()

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=64feb4f1009c1e9e60e82ac06e3a523cf2a36a9061868fc940d342e4237ee3b4
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-yanti28o
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-yanti28o
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=32ae28ac8bd68b4627bd6e792f53c0a4dfe850f8e623cfc7b108f46670d6d1e6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ah2z5fjo/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [ ]:
import os

NumUsers=33
identities={}

if __name__ == '__main__':
 for i in range(1,NumUsers+1):
  identities[i]='user_%d.npz'%(i)
  print('{} user number {}'.format(identities[i].split('.')[0],i)) 
  if not os.path.exists('SavedModel/%s'%identities[i].split('.')[0]):
   os.makedirs('SavedModel/%s'%identities[i].split('.')[0],exist_ok=True)
  if not os.path.exists('ReconstructedImages/%s'%identities[i].split('.')[0]):
   os.makedirs('ReconstructedImages/%s'%identities[i].split('.')[0],exist_ok=True)        
  ran = RAN(identities[i].split('.')[0])
  ran.train(epochs=3000, batch_size=128, save_interval=50)

Streaming output truncated to the last 5000 lines.
valid size: (64, 2, 2, 1)
1333 [D loss: 0.089269, acc.: 92.38%] [R loss: 0.002649] [R loss Val: 0.006232] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2, 2, 1)
1334 [D loss: 0.144809, acc.: 80.08%] [R loss: 0.002975] [R loss Val: 0.006764] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2, 2, 1)
1335 [D loss: 0.044715, acc.: 97.07%] [R loss: 0.003534] [R loss Val: 0.005638] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2, 2, 1)
1336 [D loss: 0.093956, acc.: 91.21%] [R loss: 0.003029] [R loss Val: 0.006471] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2, 2, 1)
1337 [D loss: 0.129921, acc.: 85.16%] [R loss: 0.002877] [R loss Val: 0.006286] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2, 2, 1)
1338 [D loss: 0.110692, acc.: 89.06%] [R loss: 0.003089] [R loss Val: 0.007193] [Minimum: 0.005462]
imgValSize: (344, 32, 32, 3)
valid size: (64, 2,

In [ ]:
import keras
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.models import load_model
import os
import numpy as np

def loss_dec_test_BC(g, data_test, data_test_gt, size):
    diffs=[]
    for i in range(1,size):
        rec_imgs = g[i].predict(data_test)
        rec_imgs = np.reshape(rec_imgs,(rec_imgs.shape[0],32*32*3))
        data_test_c = np.reshape(data_test,(data_test.shape[0],32*32*3))
        diff = np.mean(np.square(rec_imgs- data_test_c), axis=-1)
        diffs.append(diff)
    return (diffs)


data_loader = DataLoader(img_res=(32, 32))

identities={}
g={}
NUM=34


for i in range(1,NUM):
	identities[i]='user_%d.npz'%(i)
	g[i]=keras.models.load_model('SavedModel/%s/%s.h5'%(identities[i].split('.')[0],identities[i].split('.')[0]),custom_objects={'InstanceNormalization':InstanceNormalization},compile=False)
	print('The model of {} is loaded'.format(identities[i]))

 
positives=[]
for i in range(1,NUM):
        identity=identities[i]
        print(i,identity)
        
        
        imgs = data_loader.load_data(identity.split('.')[0], is_testing=True)
        imgs=np.asanyarray(imgs)
                
        
        error= loss_dec_test_BC( g,  imgs, imgs, NUM)
        error=np.asarray(error)
        positive=[]
        
        
        error_min = error.argmin(0)
        for j in range(NUM-1):
            positive.append((error_min==j).sum())
        positives.append(positive)
        print(positive) 

cm=np.asarray(positives,dtype=np.float32)
TP=np.diag(cm)
FP = np.sum(cm,axis=0) - TP
FN = np.sum(cm,axis=1) - TP
num_classes = NUM-1
TN=[]

for i in range(num_classes):
    temp = np.delete(cm,i,0)
    temp = np.delete(temp,i,1)
    TN.append(sum(sum(temp)))
recall = TP/(TP+FN)
precision = TP/(TP+FP)
print('Per user recall:\n {},\n per user precision is \n {}'.format(recall, precision))
recall = np.mean(recall)
precision = np.mean(precision)
print(' recall is\n {}, precision is\n {}' .format(recall, precision))

The model of user_1.npz is loaded
The model of user_2.npz is loaded
The model of user_3.npz is loaded
The model of user_4.npz is loaded
The model of user_5.npz is loaded
The model of user_6.npz is loaded
The model of user_7.npz is loaded
The model of user_8.npz is loaded
The model of user_9.npz is loaded
The model of user_10.npz is loaded
The model of user_11.npz is loaded
The model of user_12.npz is loaded
The model of user_13.npz is loaded
The model of user_14.npz is loaded
The model of user_15.npz is loaded
The model of user_16.npz is loaded
The model of user_17.npz is loaded
The model of user_18.npz is loaded
The model of user_19.npz is loaded
The model of user_20.npz is loaded
The model of user_21.npz is loaded
The model of user_22.npz is loaded
The model of user_23.npz is loaded
The model of user_24.npz is loaded
The model of user_25.npz is loaded
The model of user_26.npz is loaded
The model of user_27.npz is loaded
The model of user_28.npz is loaded
The model of user_29.npz is l

## 3000 epochs recall is 0.8519203662872314, precision is 0.8587789535522461

## 300 epochs 
##recall is 0.6959125995635986, precission is 0.714595377445221

## recall is 0.34601831436157227, precision is 0.3740609288215637

